# Outils de recherches sur les personnages tirés d'une document

Fichier nécessaire: un fochier en format .txt contenant la liste des noms de lieux à recenser sur la carte.

Outils disponibles
- Récupération automatique d'informations biographiques sur les personnages 
- Travail statistique sur les informations choisies : sur les dates de naissances ou le genre des personnages.

Les données concernant les personnages sont enregistrées sous forme de tableau. 

**Avant de lancer le notebook:**
- Créer un dossier sur son Google Drive intitulé "Etude-personnages".
- Placer dans ce dossier ce notebook ainsi que le fichier .txt contenant la liste des personnages.


## Connexion à son compte GoogleDrive:

- Lancer la cellule
- Cliquer sur le lien généré en dessous
Copier le mot de passe dans la nouvelle fenêtre ouverte où l'accès au compte est requis et le coller en dessous de la cellule.

In [ ]:
from google.colab import drive
import re
import itertools
import json
import urllib.parse, urllib.request, json
import collections
from collections import Counter

drive.mount('/content/drive/')
%cd /content/drive/My Drive/Etude_personnages/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/etude_personnages


## Récupération des identifiants personnages les reliant aux données issues de Wikidata:

- Changer la langue si nécessaire (par défaut: français)
- Si la détéction n'est pas assez précise, descendre "thresold" à 0.8 (moins de noms vont être identifiés), dans la cas contraire, monter le seuil à 1. Par défaut, il est placé à une valeur moyenne de 0.9
- Indiquer le chemin du fichier .txt avec la liste des personnages à la fin de la cellule suivant, au niveau de l'avant-avant dernière ligne. Ici, le fichier se nomme "liste_personnages.txt".
- Il est possible de changer le nom du fichier de sortie (fichier .txt avec liste des identifiants).

In [ ]:
%cd /content/drive/My Drive/Etude_personnages/
def CallWikifier(text, lang="fr", threshold=0.9): ### Changer ici la langue et le seuil si nécessaire
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "nqvsutgqswvfmrcvyxjtopvpiukjtp"),
        ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", ""),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
    url = "http://www.wikifier.org/annotate-article"
# Appel de wikifier pour récupérer les informations:
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 100) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
# Enregistrement des "ID" wikidata des personnages dans un fichier texte:
    with open('identifiants_personnages.txt', 'w') as outfile:
        for annotation in response["annotations"]:
          itemid = annotation["wikiDataItemId"]
          itemid2 = "".join([str(itemid)," "])
          outfile.write(itemid2)
          
with open("/content/drive/MyDrive/Etude_personnages/liste_personnages.txt", "r") as myfile:
  liste=myfile.readlines()
  CallWikifier(text=liste, lang="fr")


/content/drive/My Drive/etude_personnages


## Récupération du genre de chaque personnage identifié avec Wikidata:

- La cellule suivante récupère le genre dans lequel est classé le personnage dans Wikidata. 

- Indiquer le chemin vers le fichier texte avec les identifiantrs wikidata récupérés précédemment (ligne 7). 


In [ ]:
!pip uninstall wptools
!pip install wptools==0.1.6
import wptools

%cd /content/drive/MyDrive/Etude_personnages/

a_file = open("/content/drive/MyDrive/Etude_personnages/identifiants_personnages.txt", "r")
list_genre=[]
with open("liste_genre.txt", 'w') as outfilegenre:
  for line in a_file:
    for word in line.split(): 
      try:
        recherche_genre = wptools.page(wikibase=word, lang="fr")
        recherche_genre.WIKIPROPS['P21'] = 'gender'
        recherche_genre.get_wikidata()
        genre=recherche_genre.wikidata['gender']
      except KeyError as err:
        genre="None"
      genre2 = "".join([str(genre)," "])
      outfilegenre.write(genre2)

liste_genres = []
with open("liste_genre.txt", "r") as f:
    for line in f:
        liste_genres.extend(line.split())
        print(liste_genres)

## Récupération de toutes les informations sur les personnages :

- La cellule suivante récupère certaines informations sur les personnages : la date de naissance et de décès, le lieu de naissance et de décès.

- Toutes les informations, avec le genre récupéré dans la cellule précédente sont enregistrés sous la forme d'un fichier .json ("donnees_personnages.json") pour la suite de l'utilisation du notebook, ainsi que sous la forme d'un fichier Excel ("informations_personnages.xlsx")

In [ ]:
### Récupère les informations sur les personnages, créée un fichier json avec toutes les infos
!pip install --upgrade wptools
%cd /content/drive/MyDrive/Etude_personnages/

donnees ={}
nom = []
date_naissance_search = []
date_deces_search = []
lieu_naissance_net = []
lieu_deces_net = []
genre=[]
with open("liste_genre.txt", "r") as f:
    with open("/content/drive/MyDrive/Etude_personnages/identifiants_personnages.txt", "r") as a_file:
      with open("donnees_personnages.json", "w") as fichier : 
        for line in a_file:
          for line2 in f:
            for word, sexe in zip(line.split(), line2.split()):
              genre=str(sexe) 
              page = wptools.page(wikibase=word, lang="fr")
              page.get_wikidata()
              page.get_parse()
## Pour avoir toutes les informations:
              try:
                infobox=page.data['infobox']
                nom = infobox.get("nom")
                date_naissance=infobox.get("date de naissance")
                annee_naissance_search = re.findall(r'\b\d{3,4}\b', str(date_naissance)) ## Conserver uniquement les années
                date_deces=infobox.get("date de décès")
                annee_deces_search = re.findall(r'\b\d{3,4}\b', str(date_deces)) ### Conserver uniquement les années
                lieu_naissance = infobox.get("lieu de naissance") 
                lieu_naissance2 = re.sub(r'([()[\]{}]|)', "",str(lieu_naissance))
                lieu_naissance3 = re.sub(r'(<br>)', "",str(lieu_naissance2))
                lieu_naissance_net = re.sub(r'({{-}})', "",str(lieu_naissance3))
                lieu_deces=infobox.get("lieu de décès")
                lieu_deces2 = re.sub(r'([()[\]{}]|)', "",str(lieu_deces))
                lieu_deces3 = re.sub(r'(<br>)', "",str(lieu_deces2))
                lieu_deces_net = re.sub(r'({{-}})', "",str(lieu_deces3))
              except AttributeError as ar:
                print("Ce n'est pas un personnage!")
              donnees[nom] = {"Date de naissance" : annee_naissance_search, "Date de deces" : annee_deces_search, 
             "Lieu de naissance" : lieu_naissance_net, "Lieu de deces" : lieu_deces_net, "Genre": genre}
      
        json.dump(donnees, fichier, ensure_ascii=False)

# # Enregistrement de toutes les informations récupérées sous la forme d'un tableau Excel:
df = pd.DataFrame(donnees)
df2 = df.transpose()
df2.to_excel('informations_personnages.xlsx')

## Études statistiques sur des siècles de naissance de personnages

- Cette cellule permet :
  - d'afficher les siècles d'appartennance des personnages
  - de chercher le nombre d'occurence d'un personnage par siècle.

Les résultats s'affichent en dessous de la cellule.

In [ ]:
keys = donnees[nom]["Date de naissance"]
liste_annees = []
fichier_json = "/content/drive/MyDrive/Etude_personnages/donnees_personnages.json"
with open(fichier_json, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_annees=value["Date de naissance"]
      liste_annees.append(test_annees)
  print(liste_annees)

def calcul_siecles(liste):
  liste_siecles=[]
  for annee in liste:
    annee_net = re.sub(r'[\[\]\']', "",str(annee))
    try:
      if (int(annee_net) <= 100):
        siecle = "1"
        liste_siecles.append(siecle)
      elif (int(annee_net) % 100 == 0):
        siecle=(annee_net // 100)
        liste_siecles.append(siecle)
      else:
        siecle=(int(annee_net) // 100 + 1)
        liste_siecles.append(siecle)
    except ValueError as valerr:
      pass
  print(liste_siecles) ## Affiche les siècles correspondant aux années
  nbr_siecle=liste_siecles.count(18) ### Entrer ici le siècle cherché
  print(nbr_siecle)
  occurrences_siecles = collections.Counter(liste_siecles) ## Calcul du nombre d'occurences de chaque siècle
 

calcul_siecles(liste_annees)




[[], ['1611'], ['1638'], ['1650'], [], [], ['100', '100', '102'], [], ['272'], ['395'], [], ['1769'], ['1769'], ['1759'], ['1786']]
[17, 17, 17, 3, 4, 18, 18, 18, 18]
4


## Étude statistique sur le genre des personnages

Cette cellule permet de connaître le nombre de personnages référencés comme féminins, masculins ou "None" (c'est-à-dire non référencé) ainsi que la proportion de chaque.

Les résultats s'affichent en dessous de la cellule.



In [ ]:
keys = donnees[nom]["Genre"]
liste_des_genres = []
fichier_json = "/content/drive/MyDrive/etude_personnages/donnes_personnages.json"
with open(fichier_json, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_genres=value["Genre"]
      liste_des_genres.append(test_genres)
  print(liste_des_genres)

def proportion_genres(liste):
  c = Counter(liste)
  print(c) ## Affiche le nombre masculin / fémnin / None
  prop = [(i, c[i] / len(liste) * 100.0) for i in c]
  print(prop) ## Affiche la proportion de chaque
  
proportion_genres(liste_des_genres)


['None', 'masculin', 'masculin', 'masculin', 'None', 'None', 'masculin', 'masculin', 'masculin', 'masculin', 'None', 'masculin', 'masculin', 'masculin', 'masculin']
Counter({'masculin': 11, 'None': 4})
[('None', 26.666666666666668), ('masculin', 73.33333333333333)]
